# Introduction to the openPMD-viewer API

This notebook explains how to use the openPMD API, in order access and plot the data stored in a set of openPMD files.

## (optional) Preparing this notebook to run it locally

If you choose to run this notebook locally, you will need to download the openPMD data files which will then be visualized. To do so, execute the following cell.

In [ ]:
!wget https://github.com/RemiLehe/openPMD-example-datasets/raw/master/example-2d.tar.gz
!tar zxf example-2d.tar.gz

In addition, we choose here to incorporate the plots inside the notebook.

In [ ]:
%matplotlib inline

## Using the API

The openPMD-viewer API does not require the ipython notebook, and can be run in a normal Python environnement.
It can typically be used to write Python scripts, which perform pre-determined data analysis operations. In order to start using the API:

- Load the class `OpenPMDTimeSeries` from the module `opmd_viewer`

In [ ]:
from opmd_viewer import OpenPMDTimeSeries

- Create a time series object by pointing, to the folder which contains the corresponding openPMD data

In [ ]:
ts = OpenPMDTimeSeries('./example-2d/hdf5/')

### API for the fields

The fields can be read with the method `get_field`, by giving either the time (in seconds) or the iteration (an integer). When giving a time, it is no needed to provide the exact time of an available iteration, as the time of the closest available value will be used instead.

In [ ]:
rho, extent_rho = ts.get_field( iteration=100, field='rho' )
Ex, extent_Ex = ts.get_field( time=5.e-15,  field='E', coord='x' )

Additonal documentation on `get_field` can be obtained by reading its docstring. This can be done for instance by executing the following command in an IPython environnement.

In [ ]:
ts.get_field?

## API for the particles